# perceptron

In [27]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Load the datasets
df = pd.read_csv('../Datasets/train.csv')
test_df = pd.read_csv("../Datasets/test.csv")
y_test_df = pd.read_csv("../Datasets/gender_submission.csv")

titanic_output_for_perceptron = df['Survived']
df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
test_df = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
print(df.isna().sum())


titanic_input_for_perceptron = df
titanic_input_for_perceptron["Age"] = titanic_input_for_perceptron["Age"].fillna(titanic_input_for_perceptron["Age"].mean())
test_df["Age"] = test_df["Age"].fillna(test_df["Age"].mean())

titanic_input_for_perceptron["Embarked"] = titanic_input_for_perceptron["Embarked"].fillna(titanic_input_for_perceptron["Embarked"].mode()[0])
test_df["Embarked"] = test_df["Embarked"].fillna(test_df["Embarked"].mode()[0])


test_df["Fare"] = test_df["Fare"].fillna(test_df["Fare"].mean())

# print(titanic_input_for_perceptron.isna().sum())
print(test_df.isna().sum())

titanic_input_for_perceptron["Sex"] = titanic_input_for_perceptron['Sex'].map({"male":0, "female":1})
test_df["Sex"] = test_df['Sex'].map({"male":0, "female":1})

en = OneHotEncoder()

embarked_encoded = en.fit_transform(titanic_input_for_perceptron[['Embarked']])
embarked_encoded_test_df = en.transform(test_df[['Embarked']])

encoded_col_names = en.get_feature_names_out(['Embarked'])

titanic_input_for_perceptron[encoded_col_names] = embarked_encoded.toarray()
test_df[encoded_col_names] = embarked_encoded_test_df.toarray()

titanic_input_for_perceptron.drop('Embarked', axis=1, inplace=True)
test_df.drop('Embarked', axis=1, inplace=True)

scaler = StandardScaler()
columns = titanic_input_for_perceptron.columns  # Save column names before scaling
titanic_input_for_perceptron = pd.DataFrame(scaler.fit_transform(titanic_input_for_perceptron), columns=columns)
test_df = pd.DataFrame(scaler.transform(test_df), columns=columns)




# print(titanic_input_for_perceptron[['Fare']])


Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


In [ ]:
from sklearn.metrics import accuracy_score
import random

learning_rate = 0.01
w = [] 
epochs = 10
b = 0

# titanic_input_for_perceptron['Bias'] = 1 
# test_df['Bias'] = 1

def calc_perceptron_output(x):
    global b
    res = b
    for i, weight in enumerate(w):
        res += weight * x[i]

    return res

def update_weights(y, ye, x):
    global w, learning_rate,b
    for i, input_val in enumerate(x):
        w[i] += learning_rate * input_val * (y - ye)
    
    b += learning_rate * (y - ye)
    

def train_perceptron(inputs, outputs):
    global w

    if not w:
        w = [random.uniform(-0.01, 0.01) for _ in range(len(inputs.columns))]
    
    for e in range(epochs):
        global learning_rate
        for idx, row in inputs.iterrows():
            x = row.tolist()  
            y_true = outputs.loc[idx]     
            total = calc_perceptron_output(x)
            y_pred = 1 if total > 0 else 0 

            if y_pred != y_true:
                update_weights(y_true, y_pred, x)

            
        learning_rate = 0.1 * (1 / (1 + 0.01 * e))

def predict_y(test_inputs):
    pred_y_outputs = []

    for idx, row in test_inputs.iterrows():
        x = row.tolist()  
        total = calc_perceptron_output(x)
        y_pred = 1 if total > 0 else 0
        pred_y_outputs.append(y_pred)

    return pred_y_outputs


train_perceptron(titanic_input_for_perceptron, titanic_output_for_perceptron)

print(w)

pred_y = predict_y(test_df)

print(accuracy_score(pred_y,y_test_df[['Survived']] ))

# print(y_test_df)


[np.float64(-0.16746645637258015), np.float64(0.09896628485367892), np.float64(-0.21658610839716858), np.float64(-0.49338102096776515), np.float64(-0.048579217119302964), np.float64(0.4461619193313612), np.float64(1.4109423367164597), np.float64(0.3133278090161622), np.float64(1.072120957151573)]
0.5980861244019139
